In [56]:
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>"))

In [60]:
import importlib # использую для перегрузки библиотек
import time
from IPython.display import clear_output

import pandas as pd
import json
import numpy as np
import random

import data_preparing
import artm_model

importlib.reload(data_preparing)
importlib.reload(artm_model)
clear_output(True)

artm.version() 0.10.0


In [43]:
data_path = "service_cards_tokenised_remont_only"

In [44]:
docs = data_preparing.read_and_prepared(data_path)

In [45]:
docs.shape

(854408, 12)

In [52]:
path_train, path_test = data_preparing.form_test_train_set(docs, name="simple", test_size=0.1)

In [53]:
path_castom = data_preparing.form_castom_set(docs, name="simple")

In [61]:
model_plsa = artm_model.create_and_learn_PLSA(path_train, topic_number=750, num_collection_passes=1)

In [ ]:
model_plsa